In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
from ikrlib import *


if not os.path.isfile("SUR_projekt2022-2023.zip"):
    !wget https://www.fit.vutbr.cz/study/courses/SUR/public/projekt_2022-2023/SUR_projekt2022-2023.zip
    !unzip SUR_projekt2022-2023.zip


In [5]:
CLASSES = 31

def png2fea2(dir_name):
    """
    Loads all *.png images from directory dir_name into a dictionary. Keys are the file names
    and values and 2D numpy arrays with corresponding grayscale images
    """
    features = {}
    for f in glob(dir_name + '/*.png'):
        features[f] = np.array(Image.open(f).convert('L'), dtype=np.float64)
    return features


train_x = np.empty((0,80,80))
train_y = np.empty((0),dtype=int)

# TODO pridat vse nechci si uvarit PC
# TODO mozna bych zkonkatenoval mfcc do train_x
for i in range(1,CLASSES+1): 
    train_i = np.array(list(png2fea2('dev/'+str(i)).values()))
    label_i = np.full(len(train_i),i)
    #add to train_x
    train_x = np.concatenate((train_x, train_i), axis=0)
    train_y = np.concatenate((train_y,label_i),axis=0)


print(train_x.shape, train_y.shape)
# sample1 = train_im[0]
# print(train_im.shape)

(62, 80, 80) (62,)


In [86]:
flat = train_x.reshape(train_x.shape[0],-1)
print(flat.shape)


(12, 6400)


In [87]:
#LDA reduce dim
CLASSES = 2
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components=CLASSES-1)
lda.fit(flat,train_y)
lda_results = lda.transform(flat)
print(lda_results.shape)
print(lda_results)


(12, 1)
[[ 0.46463576]
 [-2.2421178 ]
 [-2.08799668]
 [-2.79019983]
 [-1.97974532]
 [-0.55474608]
 [ 2.15825138]
 [ 1.85282947]
 [ 0.37569382]
 [ 2.07901749]
 [ 1.73125068]
 [ 0.99312711]]


In [88]:
#train data accuarcy
(lda.predict(flat) == train_y).sum()/len(train_y)

0.9166666666666666

In [89]:
#create Test data
test_x = np.empty((0,80,80))
test_y = np.empty((0),dtype=int)

for i in range(1,CLASSES+1): 
    test_i = np.array(list(png2fea('dev/'+str(i)).values()))
    label_i = np.full(len(test_i),i)
    #add to test_x
    test_x = np.concatenate((test_x, test_i), axis=0)
    test_y = np.concatenate((test_y,label_i),axis=0)
print(test_x.shape, test_y.shape)

# flatten test data
test_x = test_x.reshape(test_x.shape[0],-1)


Processing file:  dev/1/f401_04_f18_i0_0.png
Processing file:  dev/1/f401_04_r09_i0_0.png
Processing file:  dev/2/f402_04_f18_i0_0.png
Processing file:  dev/2/f402_04_r09_i0_0.png
(4, 80, 80) (4,)


In [90]:
#test data accuarcy
print("test data accuracy", (lda.predict(test_x) == test_y).sum()/len(test_y))

0.5